## Домашнее задание в процессе выполнения...

**Практическое задание:**:
1. Взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
2. сделать обзорный анализ выбранного датасета
3. сделать feature engineering
4. обучить любой классификатор (какой вам нравится)
5. разобраться с SHAP и построить важности признаков для:
-  всего тестового набора данных (summary_plot - дать интерпретацию)
-  для топ 10% (сортировка по убыванию вероятности - таргета)
6. для отдельных наблюдений вывести force_plot и попытаться проинтерпретировать результат
7. (опциональное задание) для примера из вебинара (model A, model B) рассчитать shap-значения для точки {Fever=1, Cough=0}

**Ссылки**:
1. Пример с Permutation Importance - https://www.kaggle.com/dansbecker/permutation-importance
2. Github проекта SHAP - https://github.com/slundberg/shap
3. https://arxiv.org/pdf/1809.04559.pdf
4. https://medium.com/civis-analytics/demystifying-black-box-models-with-shap-value-analysis-3e20b536fc80
5. https://en.wikipedia.org/wiki/Shapley_value

**Feature selection**:
1. RFE - recursive feature elimination (есть в sklearn)
2. Boruta - shadow features
3. Target permutation